In [78]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

from services.twilio import client
from services.cache import get_agent_metadata, get_all_agents
from services.chat.chat import similarity_search, get_embedding
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from app.core.config import settings

In [79]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """
from composio import Composio

# Initialize the Composio client
client = Composio()

# Get the list of integrations (connected apps)
integrations = client.integrations.get()
print("Connected Apps:")
for integration in integrations:
    print(f"• App: {integration.appName}")
    print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")


Connected Apps:
• App: outlook
  ID: f70793ba-b2ea-47c2-8044-b4950c5d5e61
• App: microsoft_teams
  ID: 8339e5c5-285c-41fc-8502-2186e0587201
• App: microsoft_teams
  ID: 6a45b8a4-4298-4eca-a013-f39a83376e14
• App: googlecalendar
  ID: 76d228be-85b9-4333-9b16-fcb4a312a8cc
• App: slackbot
  ID: 4c2cf874-b381-4a4a-829a-bc6379d479a8
• App: notion
  ID: f9721737-acc8-42f9-ae18-f997370abf47
• App: notion
  ID: 77e3fc27-ae6c-46d0-83e5-398de20eb86b
• App: github
  ID: 07c756f7-1e84-491a-8d84-2f64e9da163b

Existing Connections:
• ID: 970c2459-469c-4b6b-804a-aaecd26d9c08
  App: googlecalendar
  Status: ACTIVE
• ID: d74982a6-599a-40f0-b8d3-94f5b11f20f8
  App: googlecalendar
  Status: ACTIVE
• ID: e9f2ecdf-24d9-4e32-a13e-ca539ae1fe9a
  App: outlook
  Status: INITIATED
• ID: a86b6ce2-9cc0-4795-a1cb-801377983df8
  App: outlook
  Status: INITIATED
• ID: 7ede5a6d-acf7-4b6d-9426-13dc1a4616d3
  App: notion
  Status: ACTIVE
• ID: 4db28254-01c8-4127-b99f-6caa73d22a9f
  App: outlook
  Status: ACTIVE
• ID: 3

In [80]:
from services.composio import get_calendar_slots

get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv")

TypeError: get_calendar_slots() missing 1 required positional argument: 'app'

In [75]:
from composio import Composio

# Initialize the Composio client
composio_client = Composio()

# Create a new entity with a custom ID
new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


In [77]:
""" checks if entity already exists """

existing_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")

existing_connections

Entity already exists with connections: [ConnectedAccountModel(id='970c2459-469c-4b6b-804a-aaecd26d9c08', status='ACTIVE', createdAt='2024-11-24T20:33:28.884Z', updatedAt='2024-11-24T20:33:34.161Z', appUniqueId='googlecalendar', appName='googlecalendar', integrationId='76d228be-85b9-4333-9b16-fcb4a312a8cc', connectionParams=AuthConnectionParamsModel(scope='https://www.googleapis.com/auth/calendar https://www.googleapis.com/auth/calendar.events', base_url='********', client_id='********', token_type='Bearer', access_token='********', client_secret='********', consumer_id=None, consumer_secret=None, headers=None, queryParams=None), clientUniqueUserId='user_2mmXezcGmjZCf88gT2v2waCBsXv', entityId=None), ConnectedAccountModel(id='d74982a6-599a-40f0-b8d3-94f5b11f20f8', status='ACTIVE', createdAt='2024-11-24T20:32:01.248Z', updatedAt='2024-11-24T20:32:29.205Z', appUniqueId='googlecalendar', appName='googlecalendar', integrationId='76d228be-85b9-4333-9b16-fcb4a312a8cc', connectionParams=AuthCo

[ConnectedAccountModel(id='970c2459-469c-4b6b-804a-aaecd26d9c08', status='ACTIVE', createdAt='2024-11-24T20:33:28.884Z', updatedAt='2024-11-24T20:33:34.161Z', appUniqueId='googlecalendar', appName='googlecalendar', integrationId='76d228be-85b9-4333-9b16-fcb4a312a8cc', connectionParams=AuthConnectionParamsModel(scope='https://www.googleapis.com/auth/calendar https://www.googleapis.com/auth/calendar.events', base_url='********', client_id='********', token_type='Bearer', access_token='********', client_secret='********', consumer_id=None, consumer_secret=None, headers=None, queryParams=None), clientUniqueUserId='user_2mmXezcGmjZCf88gT2v2waCBsXv', entityId=None),
 ConnectedAccountModel(id='d74982a6-599a-40f0-b8d3-94f5b11f20f8', status='ACTIVE', createdAt='2024-11-24T20:32:01.248Z', updatedAt='2024-11-24T20:32:29.205Z', appUniqueId='googlecalendar', appName='googlecalendar', integrationId='76d228be-85b9-4333-9b16-fcb4a312a8cc', connectionParams=AuthConnectionParamsModel(scope='https://www.

### Calendar Management

In [65]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App
from composio import Composio
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")

# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-24 18:51:50,471][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [70]:
tools = toolset.get_tools(actions=[Action.OUTLOOK_LIST_EVENTS])

tools

AttributeError: type object 'Action' has no attribute 'OUTLOOK_LIST_EVENTS'

In [68]:
tools = toolset.get_tools(apps=[App.OUTLOOK])

tools

[{'type': 'function',
  'function': {'name': 'OUTLOOK_OUTLOOK_CREATE_DRAFT',
   'description': 'Create a draft email using Microsoft Graph API for Outlook.',
   'parameters': {'properties': {'subject': {'description': 'Subject of the email. Please provide a value of type string. This parameter is required.',
      'examples': ['Team Meeting'],
      'maxLength': 255,
      'minLength': 1,
      'title': 'Subject',
      'type': 'string'},
     'body': {'description': 'Body content of the email. Can be plain text or HTML based on is_html flag. Please provide a value of type string. This parameter is required.',
      'examples': ["Let's meet to discuss the project."],
      'title': 'Body',
      'type': 'string'},
     'to_recipients': {'description': 'List of recipient email addresses. This parameter is required.',
      'examples': [['johndoe@example.com', 'janedoe@example.com']],
      'items': {'type': 'string'},
      'title': 'To Recipients',
      'type': 'array'},
     'cc_reci

In [62]:
openai_client = OpenAI()

today = datetime.now().strftime("%Y-%m-%d")
task = f"get calendar slots for next week, today is {today}"

# Update the tools to include search capability
tools = toolset.get_tools(actions=[
    Action.OUTLOOK_OUTLOOK_UPDATE_CALENDAR_EVENT])


# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = toolset.handle_tool_calls(search_response)
search_result


In [63]:
search_result

[]

### Shopify

In [ ]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, App

toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY)

print(f"Open this URL to authenticate: {request.redirectUrl}")


### Notion

In [38]:
"""Auth"""

from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-24 02:13:41,224][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL to authenticate: https://api.notion.com/v1/oauth/authorize?client_id=17eb9b75-7ad0-4fc1-b01d-12bb09271484&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=&response_type=code&owner=user&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiI3ZWRlNWE2ZC1hY2Y3LTRiNmQtOTQyNi0xM2RjMWE0NjE2ZDMiLCJpbnRlZ3JhdGlvbklkIjoiZjk3MjE3MzctYWNjOC00MmY5LWFlMTgtZjk5NzM3MGFiZjQ3IiwiYXBwTmFtZSI6Im5vdGlvbiIsImNsaWVudEluZm8iOnsiY2xpZW50SWQiOiIwMTI3MTQ1OS04NjNmLTQ3NzAtYThhMy0xNjZhY2I3ODFmMzciLCJtZW1iZXJJZCI6IjJiZDI2MDVkLTJiOTgtNDBlOC1iYzU2LWY4ZDY2YTE1NjllYyJ9LCJpYXQiOjE3MzI0MTQ0MjJ9.MGqznQrMQaONYyL-xMbFLE1bizfNc9Eh-76rEhpMWSU&code_challenge=QK1DSIfYRAQgkR_gpLfvqZFPbZFJnSAh1V5qR-ml2YE&code_challenge_method=S256


In [3]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")

[2024-11-23 04:32:28,947][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [16]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



[2024-11-23 04:57:43,155][INFO] Executing `NOTION_SEARCH_NOTION_PAGE` with params={'query': 'DevOps DB', 'filter_value': 'database'} and metadata={} connected_account_id=None
[2024-11-23 04:57:46,528][INFO] Got response={'successfull': True, 'data': {'response_data': {'object': 'list', 'results': [{'object': 'database', 'id': '11b53e39-0487-804b-bca4-dd524767939b', 'cover': {'type': 'external', 'external': {'url': 'https://www.notion.so/images/page-cover/nasa...


DB ID: 11b53e39-0487-804b-bca4-dd524767939b


In [17]:
print(search_result)


[]


### Google Calendar

In [89]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


GOOGLECALENDAR_CREATE_EVENT: 'Action'
GOOGLECALENDAR_DELETE_EVENT: 'Action'
GOOGLECALENDAR_DUPLICATE_CALENDAR: 'Action'
GOOGLECALENDAR_FIND_EVENT: 'Action'
GOOGLECALENDAR_FIND_FREE_SLOTS: 'Action'
GOOGLECALENDAR_GET_CALENDAR: 'Action'
GOOGLECALENDAR_GET_CURRENT_DATE_TIME: 'Action'
GOOGLECALENDAR_LIST_CALENDARS: 'Action'
GOOGLECALENDAR_PATCH_CALENDAR: 'Action'
GOOGLECALENDAR_QUICK_ADD: 'Action'
GOOGLECALENDAR_REMOVE_ATTENDEE: 'Action'
GOOGLECALENDAR_UPDATE_EVENT: 'Action'

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_LIST_CALENDARS])
# task = "find free slots for next week"


# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_GET_CALENDAR])
# task = "fetch calendar for id michael@flowon.ai"

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_FIND_FREE_SLOTS])
task = "fetch calendar for id michael@flowon.ai. "


today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


[2024-11-25 05:03:12,222][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


[2024-11-25 05:03:15,982][INFO] Executing `GOOGLECALENDAR_FIND_FREE_SLOTS` with params={'items': ['michael@flowon.ai']} and metadata={} connected_account_id=None
[2024-11-25 05:03:17,607][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#freeBusy', 'timeMin': '2024-11-25T00:00:00.000Z', 'timeMax': '2024-11-25T23:59:59.000Z', 'calendars': {'michael@flowon.ai': {'busy': [{'start': '2024-11-25T12:30:00Z', 'end': '...


[{'successfull': True, 'data': {'response_data': {'kind': 'calendar#freeBusy', 'timeMin': '2024-11-25T00:00:00.000Z', 'timeMax': '2024-11-25T23:59:59.000Z', 'calendars': {'michael@flowon.ai': {'busy': [{'start': '2024-11-25T12:30:00Z', 'end': '2024-11-25T13:30:00Z'}, {'start': '2024-11-25T14:00:00Z', 'end': '2024-11-25T14:30:00Z'}, {'start': '2024-11-25T15:00:00Z', 'end': '2024-11-25T15:30:00Z'}]}}}}, 'error': None}]


In [5]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Create a 1 hour meeting event at 5:30PM tomorrow regarding OpenAI Integration"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


[2024-11-23 04:40:44,386][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'calendar_id': 'primary', 'create_meeting_room': True, 'event_duration': '1h', 'start_datetime': '2024-11-24T17:30:00', 'summary': 'OpenAI Integration Meeting'} and metadata={} connected_account_id=None
[2024-11-23 04:40:46,334][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3464673691676000"', 'id': 'i62bupprdrf2veopdjo7l7629k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aTYyYnVwcHJkcmYydmVvcGRq...


[{'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3464673691676000"', 'id': 'i62bupprdrf2veopdjo7l7629k', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aTYyYnVwcHJkcmYydmVvcGRqbzdsNzYyOWsgbWljaGFlbEBmbG93b24uYWk', 'created': '2024-11-23T04:40:45.000Z', 'updated': '2024-11-23T04:40:45.838Z', 'summary': 'OpenAI Integration Meeting', 'creator': {'email': 'michael@flowon.ai', 'self': True}, 'organizer': {'email': 'michael@flowon.ai', 'self': True}, 'start': {'dateTime': '2024-11-24T17:30:00Z', 'timeZone': 'UTC'}, 'end': {'dateTime': '2024-11-24T18:30:00Z', 'timeZone': 'UTC'}, 'iCalUID': 'i62bupprdrf2veopdjo7l7629k@google.com', 'sequence': 0, 'hangoutLink': 'https://meet.google.com/egu-qfzk-xcg', 'conferenceData': {'createRequest': {'requestId': '0a06f1a31bcb4b8fb75d8144fc6848fb', 'conferenceSolutionKey': {'type': 'hangoutsMeet'}, 'status': {'statusCode': 'success'}}, 'entryPoints': [{'entryPointType': 'video', 'uri': 'http